In [117]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import csv
import random
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from scipy.stats import linregress
import random
import re

In [118]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/whats-cooking-kernels-only/sample_submission.csv.zip
/kaggle/input/whats-cooking-kernels-only/train.json
/kaggle/input/whats-cooking-kernels-only/test.json


In [119]:
# Open the JSON file
with open('/kaggle/input/whats-cooking-kernels-only/train.json', 'r') as f:
    # Load the JSON data into a Python dictionary
    train_data = json.load(f)

with open('/kaggle/input/whats-cooking-kernels-only/test.json', 'r') as f:
    # Load the JSON data into a Python dictionary
    test_data = json.load(f)    


In [120]:
def check_null(data):
    # Check if any field contains a null value
    contains_null = False
    for j in data:
        data = j
        for key, value in data.items():
            if value is None:
                contains_null = True
                break

    # Print the result
    if contains_null:
        print("The JSON object contains a null value")
    else:
        print("The JSON object does not contain a null value")        

check_null(train_data)
check_null(test_data)


The JSON object does not contain a null value
The JSON object does not contain a null value


In [121]:
# constants
EMBEDDING_DIM = 100
MAXLEN = 65
TRUNCATING = 'post'
PADDING = 'post'
OOV_TOKEN = "<OOV>"
MAX_EXAMPLES = 160000
TRAINING_SPLIT = 0.9

In [122]:
train_labels = []
train_ingredients = []

test_ingredients = []
test_ids = []

train_data = random.sample(train_data, len(train_data))

for j in train_data:
    train_labels.append(j['cuisine'])
    train_ingredients.append(j['ingredients'])    
    
for j in test_data:
    test_ingredients.append(j['ingredients'])
    test_ids.append(j['id'])
    
all_labels = train_labels.copy()

In [123]:
print(len(test_ids))

9944


In [124]:
# Find the length of the biggest ingrediants list
max_len = max([len(lst) for lst in train_ingredients])

# Print the result
print("The length of the biggest list is:", max_len)
MAXLEN = 65

The length of the biggest list is: 65


In [125]:
def split_data(input_data, split_ratio):
    input_data_split_size = int(len(input_data) * split_ratio)
    train_data = input_data[0:input_data_split_size]
    val_data = input_data[input_data_split_size:]
    
    return train_data, val_data

train_ingredients, val_ingredients = split_data(train_ingredients, TRAINING_SPLIT)
train_labels, val_labels = split_data(train_labels, TRAINING_SPLIT)

In [126]:
def fit_tokenizer(train_sentences, oov_token):
    
    ### START CODE HERE
    
    # Instantiate the Tokenizer class, passing in the correct values for num_words and oov_token
    tokenizer = Tokenizer(oov_token=oov_token)
    
    # Fit the tokenizer to the training sentences
    tokenizer.fit_on_texts(train_sentences)
    
    ### END CODE HERE
    
    return tokenizer

tokenizer_ingredients = fit_tokenizer(train_ingredients, OOV_TOKEN)
word_index_ingredients = tokenizer_ingredients.word_index
VOCAB_SIZE = len(word_index_ingredients)

print(f"Vocabulary contains {VOCAB_SIZE} words\n")
print("<OOV> token included in vocabulary" if "<OOV>" in word_index_ingredients else "<OOV> token NOT included in vocabulary")

Vocabulary contains 6517 words

<OOV> token included in vocabulary


In [127]:
def tokenize_labels(all_labels, split_labels):
    
    ### START CODE HERE
    
    # Instantiate the Tokenizer (no additional arguments needed)
    label_tokenizer = Tokenizer(filters=" ")
    
    # Fit the tokenizer on all the labels
    label_tokenizer.fit_on_texts(all_labels)
    
    # Convert labels to sequences
    label_seq = label_tokenizer.texts_to_sequences(split_labels)
    print(label_tokenizer.word_index)
    # Convert sequences to a numpy array. Don't forget to substact 1 from every entry in the array!
    label_seq_np = np.array(label_seq) - 1
    
    ### END CODE HERE
    
    return label_seq_np, label_tokenizer

train_label_seq, train_label_tokenizer = tokenize_labels(all_labels, train_labels)
val_label_seq, val_label_tokenizer = tokenize_labels(all_labels, val_labels)

{'italian': 1, 'mexican': 2, 'southern_us': 3, 'indian': 4, 'chinese': 5, 'french': 6, 'cajun_creole': 7, 'thai': 8, 'japanese': 9, 'greek': 10, 'spanish': 11, 'korean': 12, 'vietnamese': 13, 'moroccan': 14, 'british': 15, 'filipino': 16, 'irish': 17, 'jamaican': 18, 'russian': 19, 'brazilian': 20}
{'italian': 1, 'mexican': 2, 'southern_us': 3, 'indian': 4, 'chinese': 5, 'french': 6, 'cajun_creole': 7, 'thai': 8, 'japanese': 9, 'greek': 10, 'spanish': 11, 'korean': 12, 'vietnamese': 13, 'moroccan': 14, 'british': 15, 'filipino': 16, 'irish': 17, 'jamaican': 18, 'russian': 19, 'brazilian': 20}


In [128]:
def seq_pad_and_trunc(sentences, tokenizer, padding, truncating, maxlen):
    
    ### START CODE HERE
       
    # Convert sentences to sequences
    sequences = tokenizer.texts_to_sequences(sentences)
    
    # Pad the sequences using the correct padding, truncating and maxlen
#     pad_trunc_sequences = pad_sequences(sequences, maxlen=maxlen, padding=padding, truncating=truncating)
    pad_trunc_sequences = pad_sequences(sequences, padding=padding, truncating=truncating)
    
    ### END CODE HERE
    
    return pad_trunc_sequences


In [129]:
# Test your function
train_pad_trunc_seq = seq_pad_and_trunc(train_ingredients, tokenizer_ingredients, PADDING, TRUNCATING, MAXLEN)
val_pad_trunc_seq = seq_pad_and_trunc(val_ingredients, tokenizer_ingredients, PADDING, TRUNCATING, MAXLEN)

test_pad_trunc_seq = seq_pad_and_trunc(test_ingredients, tokenizer_ingredients, PADDING, TRUNCATING, MAXLEN)

print(f"Padded and truncated training sequences have shape: {train_pad_trunc_seq.shape}\n")
print(f"Padded and truncated validation sequences have shape: {val_pad_trunc_seq.shape}")
print(f"Padded and truncated test sequences have shape: {test_pad_trunc_seq.shape}")

Padded and truncated training sequences have shape: (35796, 65)

Padded and truncated validation sequences have shape: (3978, 36)
Padded and truncated test sequences have shape: (9944, 50)


In [130]:
# GRADED FUNCTION: create_model
def create_model(num_words, embedding_dim, maxlen):
    
    tf.random.set_seed(123)
    
    ### START CODE HERE
    
    model = tf.keras.Sequential([ 
#         tf.keras.layers.Embedding(num_words, embedding_dim, input_length=maxlen),
        tf.keras.layers.Embedding(num_words, embedding_dim),    
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dense(24, activation='relu'),        
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(20, activation='softmax')
    ])
    
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy']) 

    ### END CODE HERE

    return model

In [131]:
# train_label_seq = train_label_seq.reshape(35796,)
# val_label_seq = val_label_seq.reshape(3978,)
train_label_seq = train_label_seq.reshape(-1,)
val_label_seq = val_label_seq.reshape(-1,)
print(train_pad_trunc_seq.shape)
print(train_label_seq.shape)
print(val_pad_trunc_seq.shape)
print(val_label_seq.shape)

(35796, 65)
(35796,)
(3978, 36)
(3978,)


In [132]:
model = create_model(6544, EMBEDDING_DIM, MAXLEN)

history = model.fit(train_pad_trunc_seq, train_label_seq, epochs=30, validation_data=(val_pad_trunc_seq, val_label_seq))

Epoch 1/30
1119/1119 [==============================] - 10s 8ms/step - loss: 2.0912 - accuracy: 0.3657 - val_loss: 1.4272 - val_accuracy: 0.5601
Epoch 2/30
1119/1119 [==============================] - 9s 8ms/step - loss: 1.3346 - accuracy: 0.6061 - val_loss: 1.2188 - val_accuracy: 0.6433
Epoch 3/30
1119/1119 [==============================] - 9s 8ms/step - loss: 1.1430 - accuracy: 0.6606 - val_loss: 1.1561 - val_accuracy: 0.6657
Epoch 4/30
1119/1119 [==============================] - 9s 8ms/step - loss: 1.0436 - accuracy: 0.6869 - val_loss: 1.1237 - val_accuracy: 0.6795
Epoch 5/30
1119/1119 [==============================] - 9s 8ms/step - loss: 0.9729 - accuracy: 0.7059 - val_loss: 1.1015 - val_accuracy: 0.6938
Epoch 6/30
1119/1119 [==============================] - 9s 8ms/step - loss: 0.9191 - accuracy: 0.7239 - val_loss: 1.1233 - val_accuracy: 0.6971
Epoch 7/30
1119/1119 [==============================] - 9s 8ms/step - loss: 0.8789 - accuracy: 0.7379 - val_loss: 1.1412 - val_accuracy

In [133]:
softmax_outputs = model.predict(test_pad_trunc_seq)
print(softmax_outputs)

311/311 [==============================] - 1s 2ms/step
[[8.6246962e-03 2.3304052e-03 1.2796967e-01 ... 1.3136910e-02
  9.8664045e-02 1.7717783e-03]
 [8.1992644e-09 7.4570835e-07 9.9993449e-01 ... 2.2835459e-06
  5.0929287e-07 2.9816121e-07]
 [7.4887878e-01 6.9818072e-02 4.2321739e-05 ... 1.0488388e-09
  1.1921193e-05 1.1842345e-04]
 ...
 [5.5893809e-01 2.7153874e-04 2.2434574e-01 ... 1.3001803e-05
  1.9612356e-05 3.2609933e-07]
 [1.2909155e-07 2.1343961e-04 9.9974382e-01 ... 3.4674192e-09
  2.2969827e-10 1.7950633e-08]
 [8.3842791e-14 9.9999994e-01 5.9348793e-09 ... 2.2616626e-24
  1.8999456e-25 1.6891812e-08]]


In [134]:
softmax_indices = np.argmax(softmax_outputs, axis=1)
len(softmax_indices)

9944

In [135]:
train_label_tokenizer.index_word

{1: 'italian',
 2: 'mexican',
 3: 'southern_us',
 4: 'indian',
 5: 'chinese',
 6: 'french',
 7: 'cajun_creole',
 8: 'thai',
 9: 'japanese',
 10: 'greek',
 11: 'spanish',
 12: 'korean',
 13: 'vietnamese',
 14: 'moroccan',
 15: 'british',
 16: 'filipino',
 17: 'irish',
 18: 'jamaican',
 19: 'russian',
 20: 'brazilian'}

In [136]:
print(len(test_ids))
predictions = []
for idx in softmax_indices:
    predictions.append(train_label_tokenizer.index_word[idx+1])

predictions_dict = {
    'id' : test_ids,
    'cuisine' : predictions
}
predictions_df = pd.DataFrame(predictions_dict)


9944


In [137]:
predictions_df.to_csv('submission.csv', index=False)